In [56]:
%run ./_api_keys
%run ./fred

In [2]:
import fredapi
import pandas as pd
from tqdm import tqdm
from _api_keys import get_fred_api_key

Fred = fredapi.Fred(api_key=get_fred_api_key())

In [65]:
df = pd.read_clipboard()
df.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,ELF,CPREP,...,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
0,1,AL,0,1,785000,519980,85690,165290,724170,22560,...,0,0,57720,46577,674840,1827202,672200,1818867,2900,6089
1,1,AL,0,2,554310,270870,121420,146470,515150,13260,...,0,0,81770,112540,470410,1445383,466960,1432458,4660,11648
2,1,AL,0,3,290630,113280,124770,44570,269700,6420,...,0,0,70360,144380,220710,626662,216530,610170,5760,16235
3,1,AL,0,4,181010,42010,120820,14410,168830,2570,...,0,0,49500,135429,130670,437179,126790,419324,3730,14903
4,1,AL,0,5,269080,31310,224330,8270,252360,3250,...,100,20,103250,470206,165650,724529,156910,642895,11280,80064


In [66]:
df.to_parquet('./irs_zip_code_data_2020.parquet')

In [55]:
treasury_series = pd.read_excel("./fred_treasury_series_names.xlsx")
treasury_series.head()

,treasury_series
0,T10Y2Y
1,T10Y2YM
2,T10Y3M
3,DFII10
4,T10Y3MM


In [57]:
df = get_fred_data('T10Y2Y', frequency='d')
df.head()

2016-01-01     NaN
2016-01-04    1.22
2016-01-05    1.21
2016-01-06    1.19
2016-01-07    1.20
dtype: float64

In [59]:
rawdf = Fred.search('T10Y2Y')
rawdf.to_clipboard()

In [54]:
df = rawdf.copy()
df.to_clipboard()

* Consumer Price Index for All Urban Consumers: Used Cars and Trucks in U.S. City Average (CUSR0000SETA02)
* Sticky Price Consumer Price Index (STICKCPIM157SFRBATL)		
  * The Sticky Price Consumer Price Index (CPI) is calculated from a subset of goods and services included in the CPI that change price relatively infrequently. Because these goods and services change price relatively infrequently, they are thought to incorporate expectations about future inflation to a greater degree than prices that change on a more frequent basis. One possible explanation for sticky prices could be the costs firms incur when changing price.
* Sticky Price Consumer Price Index less Food and Energy (CORESTICKM159SFRBATL)
* Median Consumer Price Index (MEDCPIM158SFRBCLE)
* Consumer Price Index: All Items for the United States (USACPIALLMINMEI)	
* Consumer Price Index for All Urban Consumers: New Vehicles in U.S. City Average (CUSR0000SETA01)	
* Consumer Price Index for All Urban Consumers: Shelter in U.S. City Average (CUSR0000SAH1)	
* Consumer Price Index for All Urban Consumers: Food in U.S. City Average (CPIUFDSL)	
* Consumer Price Index for All Urban Consumers: Energy in U.S. City Average (CPIENGSL)
* Consumer Price Index for All Urban Consumers: Electricity in U.S. City Average (CUSR0000SEHF01)	
* Consumer Price Index for All Urban Consumers: Apparel in U.S. City Average (CPIAPPSL)	
* Consumer Price Index for All Urban Consumers: Services Less Energy Services in U.S. City Average (CUSR0000SASLE)	
* Consumer Price Index for All Urban Consumers: Medical Care Services in U.S. City Average (CUSR0000SAM2)	
* Consumer Price Index for All Urban Consumers: Gasoline (All Types) in U.S. City Average (CUSR0000SETB01)	
* Consumer Price Index for All Urban Consumers: Food Away from Home in U.S. City Average (CUSR0000SEFV)	
* Consumer Price Index for All Urban Consumers: Airline Fares in U.S. City Average (CUSR0000SETG01)	
* Consumer Price Index for All Urban Consumers: Medical Care Commodities in U.S. City Average (CUSR0000SAM1)	
* Consumer Price Index for All Urban Consumers: Education and Communication in U.S. City Average (CPIEDUSL)	
* Consumer Price Index for All Urban Consumers: Owners' Equivalent Rent of Residences in U.S. City Average (CUSR0000SEHC)
* Consumer Price Index for All Urban Consumers: Housing in U.S. City Average (CPIHOSSL)	
* Consumer Price Index for All Urban Consumers: Medical Care in U.S. City Average (CPIMEDSL)	
* Consumer Price Index for All Urban Consumers: Motor Vehicle Maintenance and Repair in U.S. City Average (CUSR0000SETD)	

	


In [3]:
def get_fred_data(series_id: str,
                    start_date: str ='2016-01-01',
                    end_date: str = None,
                    frequency: str = 'd') -> pd.DataFrame:
        """
        Get data from FRED and return as a pandas DataFrame.
    
        Parameters
        ----------
        series_id : str
            The series id of the data to get.
        start_date : str
            The start date of the data to get, by default '2016-01-01'.
        end_date : str, optional
            The end date of the data to get, by default None. If None, the
            current date is used.
        frequency : str, optional
            The frequency of the data to get, by default 'd'.
    
        Returns
        -------
        pd.DataFrame
            The data from FRED as a pandas DataFrame.
        """
        if end_date is None:
            end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
        return Fred.get_series(series_id,
                             observation_start=start_date,
                             observation_end=end_date,
                             frequency=frequency)

In [5]:
unemployment = pd.read_parquet('./fred_unemployment.parquet')
state_unemployment_list = []
state_series = unemployment.loc[unemployment.state_indicator.eq(1), 'id'].drop_duplicates().tolist()
for s in tqdm(state_series):
    test = get_fred_data(s, frequency='m').reset_index().assign(id=s).set_index(['id']).rename(columns={'index': 'date', 0: 'state_unemployment_rate'})
    state_unemployment_list.append(test)
state_unemployment = pd.concat(state_unemployment_list)
state_unemployment.to_parquet('./fred_state_unemployment_pull.parquet')

county_unemployment_list = []
county_series = unemployment.loc[unemployment.county_indicator.eq(1), 'id'].drop_duplicates().tolist()
for s in tqdm(county_series):
    test = get_fred_data(s, frequency='m').reset_index().assign(id=s).set_index(['id']).rename(columns={'index': 'date', 0: 'county_unemployment_rate'})
    county_unemployment_list.append(test)
county_unemployment = pd.concat(county_unemployment_list)
county_unemployment.to_parquet('./fred_county_unemployment_pull.parquet')

msa_unemployment_list = []
msa_series = unemployment.loc[unemployment.msa_indicator.eq(1), 'id'].drop_duplicates().tolist()
for s in tqdm(msa_series):
    test = get_fred_data(s, frequency='m').reset_index().assign(id=s).set_index(['id']).rename(columns={'index': 'date', 0: 'msa_unemployment_rate'})
    msa_unemployment_list.append(test)
msa_unemployment = pd.concat(msa_unemployment_list)
msa_unemployment.to_parquet('./fred_msa_unemployment_pull.parquet')

county = pd.read_parquet('./fred_county_unemployment_pull.parquet').rename(columns={'county_unemployment_rate': 'unemployment_rate'})
msa = pd.read_parquet('./fred_msa_unemployment_pull.parquet').rename(columns={'msa_unemployment_rate': 'unemployment_rate'})
state = pd.read_parquet('./fred_state_unemployment_pull.parquet').rename(columns={'state_unemployment_rate': 'unemployment_rate'})
pull = pd.concat([county, msa, state])
unemployment = pd.read_parquet('./fred_unemployment.parquet')\
                .reset_index()\
                .drop(columns='series id')\
                .set_index('id')\
                .join(pull, how='left')\
                .reset_index()

county_unemployment = unemployment\
                        .loc[unemployment.county_indicator.eq(1), ['location', 'date', 'unemployment_rate']]\
                        .rename(columns={'location': 'county', 'unemployment_rate': 'county_unemployment_rate'})\
                        .reset_index(drop=True)\
                        .assign(state=lambda x: x.county.str.split(',').str[1].str.strip(),
                                county=lambda x: x.county.str.split(',').str[0].str.strip())\
                        .assign(county=lambda x: x.county.str.replace(' county', ''))
county_unemployment.to_parquet('./fred_county_unemployment.parquet')

msa_unemployment = unemployment\
                        .loc[unemployment.msa_indicator.eq(1), ['location', 'date', 'unemployment_rate']]\
                        .rename(columns={'location': 'msa', 'unemployment_rate': 'msa_unemployment_rate'})\
                        .reset_index(drop=True)\
                        .assign(state=lambda x: x.msa.str.split(',').str[1].str.strip(),
                                msa=lambda x: x.msa.str.split(',').str[0].str.strip())\
                        .assign(msa=lambda x: x.msa.str.replace(' MSA', ''))
msa_unemployment['state'] = msa_unemployment['state'].str.replace(' (msa)', '')
msa_unemployment.to_parquet('./fred_msa_unemployment.parquet')

state_unemployment = unemployment\
                        .loc[unemployment.state_indicator.eq(1), ['location', 'date', 'unemployment_rate']]\
                        .rename(columns={'location': 'state', 'unemployment_rate': 'state_unemployment_rate'})\
                        .reset_index(drop=True)\
                        .assign(state=lambda x: x.state.str.strip())
state_unemployment.to_parquet('./fred_state_unemployment.parquet')

100%|██████████| 65/65 [00:35<00:00,  1.81it/s]


In [10]:
county = pd.read_parquet('./fred_county_unemployment_pull.parquet')
county

,date,county_unemployment_rate
id,,
PAADAM0URN,2016-01-01,4.3
PAADAM0URN,2016-02-01,4.3
PAADAM0URN,2016-03-01,4.0
PAADAM0URN,2016-04-01,3.5
PAADAM0URN,2016-05-01,3.7
...,...,...
PAYORK0URN,2023-02-01,3.9
PAYORK0URN,2023-03-01,3.1
PAYORK0URN,2023-04-01,2.7


In [30]:
county = pd.read_parquet('./fred_county_unemployment_pull.parquet').rename(columns={'county_unemployment_rate': 'unemployment_rate'})
msa = pd.read_parquet('./fred_msa_unemployment_pull.parquet').rename(columns={'msa_unemployment_rate': 'unemployment_rate'})
state = pd.read_parquet('./fred_state_unemployment_pull.parquet').rename(columns={'state_unemployment_rate': 'unemployment_rate'})
pull = pd.concat([county, msa, state])
unemployment = pd.read_parquet('./fred_unemployment.parquet')\
                .reset_index()\
                .drop(columns='series id')\
                .set_index('id')\
                .join(pull, how='left')\
                .reset_index()
unemployment

,id,title,units,location,msa_indicator,county_indicator,state_indicator,date,unemployment_rate
0,AKURN,unemployment rate in alaska,percent,alaska,0,0,1,2016-01-01,7.0
1,AKURN,unemployment rate in alaska,percent,alaska,0,0,1,2016-02-01,7.3
2,AKURN,unemployment rate in alaska,percent,alaska,0,0,1,2016-03-01,7.1
3,AKURN,unemployment rate in alaska,percent,alaska,0,0,1,2016-04-01,6.8
4,AKURN,unemployment rate in alaska,percent,alaska,0,0,1,2016-05-01,6.4
...,...,...,...,...,...,...,...,...,...
23035,WYURN,unemployment rate in wyoming,percent,wyoming,0,0,1,2023-02-01,4.0
23036,WYURN,unemployment rate in wyoming,percent,wyoming,0,0,1,2023-03-01,4.1
23037,WYURN,unemployment rate in wyoming,percent,wyoming,0,0,1,2023-04-01,3.3
23038,WYURN,unemployment rate in wyoming,percent,wyoming,0,0,1,2023-05-01,2.7


In [39]:
county_unemployment = unemployment\
                        .loc[unemployment.county_indicator.eq(1), ['location', 'date', 'unemployment_rate']]\
                        .rename(columns={'location': 'county', 'unemployment_rate': 'county_unemployment_rate'})\
                        .reset_index(drop=True)\
                        .assign(state=lambda x: x.county.str.split(',').str[1].str.strip(),
                                county=lambda x: x.county.str.split(',').str[0].str.strip())\
                        .assign(county=lambda x: x.county.str.replace(' county', ''))
county_unemployment.to_parquet('./fred_county_unemployment.parquet')

In [46]:
msa_unemployment = unemployment\
                        .loc[unemployment.msa_indicator.eq(1), ['location', 'date', 'unemployment_rate']]\
                        .rename(columns={'location': 'msa', 'unemployment_rate': 'msa_unemployment_rate'})\
                        .reset_index(drop=True)\
                        .assign(state=lambda x: x.msa.str.split(',').str[1].str.strip(),
                                msa=lambda x: x.msa.str.split(',').str[0].str.strip())\
                        .assign(msa=lambda x: x.msa.str.replace(' MSA', ''))
msa_unemployment['state'] = msa_unemployment['state'].str.replace(' (msa)', '')
msa_unemployment.to_parquet('./fred_msa_unemployment.parquet')

In [41]:
state_unemployment = unemployment\
                        .loc[unemployment.state_indicator.eq(1), ['location', 'date', 'unemployment_rate']]\
                        .rename(columns={'location': 'state', 'unemployment_rate': 'state_unemployment_rate'})\
                        .reset_index(drop=True)\
                        .assign(state=lambda x: x.state.str.strip())
state_unemployment.to_parquet('./fred_state_unemployment.parquet')

In [32]:
import featuretools as ft

ModuleNotFoundError: No module named 'featuretools'

In [31]:
unemployment.units.drop_duplicates()

0    percent
Name: units, dtype: object

In [15]:
df = fred.search('unemployment')
df['count'] = df.groupby('title')['title'].transform('count')

df = df.copy().loc[df['title'].str.contains("Unemployment Rate in")]
df = df.loc[df.frequency_short.eq('M')]
df = df.loc[df.seasonal_adjustment.eq('Not Seasonally Adjusted')]

df['location'] = df['title'].str.split(' in ').str[1]
df = df.loc[~df['location'].str.lower().str.contains('census')]


df['msa_indicator'] = df['location'].str.lower().str.contains('msa').astype(int)

extra_msas = ['Orleans Parish, LA',
              'Providence-Warwick',
              'RI-MA (NECTA)',
              'St. Louis City, MO',
              'Danville City, VA',
              'Boston-Cambridge-Nashua, MA-NH (NECTA)']

df['msa_indicator'] = df.msa_indicator.mask(df.location.isin(extra_msas), 1)
df['msa_indicator'] = df.msa_indicator.mask(df.title.eq("Unemployment Rate in Providence-Warwick, RI-MA (NECTA)"), 1)
df['msa_indicator'] = df.msa_indicator.mask(df.title.eq("Unemployment Rate in Baltimore City, MD"), 1)
df['msa_indicator'] = df.msa_indicator.mask(df.title.eq("Unemployment Rate in the District of Columbia"), 1)



df['county_indicator'] = df['location'].str.lower().str.contains('county').astype(int)
df['county_indicator'] = df['county_indicator'].mask(df['msa_indicator'].eq(1), 0)

df['state_indicator'] = 0
df.loc[df.msa_indicator.eq(0) & df.county_indicator.eq(0), 'state_indicator'] = 1

unemployment_data = df[['title', 'location', 'msa_indicator', 'county_indicator', 'state_indicator', 'id']].copy()

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes,count
series id,,,,,,,,,,,,,,,,
UNRATE,UNRATE,2023-08-18,2023-08-18,Unemployment Rate,1948-01-01,2023-07-01,Monthly,M,Percent,%,Seasonally Adjusted,SA,2023-08-04 07:45:03-05:00,92,The unemployment rate represents the number of...,2
UNRATENSA,UNRATENSA,2023-08-18,2023-08-18,Unemployment Rate,1948-01-01,2023-07-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-08-04 07:45:34-05:00,53,The unemployment rate represents the number of...,2
CCSA,CCSA,2023-08-18,2023-08-18,Continued Claims (Insured Unemployment),1967-01-07,2023-08-05,"Weekly, Ending Saturday",W,Number,Number,Seasonally Adjusted,SA,2023-08-17 07:33:03-05:00,74,"Continued claims, also referred to as insured ...",2
CCNSA,CCNSA,2023-08-18,2023-08-18,Continued Claims (Insured Unemployment),1967-01-07,2023-08-05,"Weekly, Ending Saturday",W,Number,Number,Not Seasonally Adjusted,NSA,2023-08-17 07:33:03-05:00,41,"Continued claims, also referred to as insured ...",2
UNEMPLOY,UNEMPLOY,2023-08-18,2023-08-18,Unemployment Level,1948-01-01,2023-07-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2023-08-04 07:45:25-05:00,65,The series comes from the 'Current Population ...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OHMADI7URN,OHMADI7URN,2023-08-18,2023-08-18,"Unemployment Rate in Madison County, OH",1990-01-01,2023-06-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-08-02 10:19:54-05:00,5,These data come from the Current Population Su...,1
ILLAKE7URN,ILLAKE7URN,2023-08-18,2023-08-18,"Unemployment Rate in Lake County, IL",1990-01-01,2023-06-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-08-02 10:19:03-05:00,5,These data come from the Current Population Su...,1
PAADAM0URN,PAADAM0URN,2023-08-18,2023-08-18,"Unemployment Rate in Adams County, PA",1990-01-01,2023-06-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-08-02 10:13:12-05:00,5,These data come from the Current Population Su...,1


In [47]:
unemployment_data = df[['title', 'units', 'location', 'msa_indicator', 'county_indicator', 'state_indicator', 'id']].copy().drop_duplicates()
unemployment_data['title'] = unemployment_data['title'].str.lower()
unemployment_data['units'] = unemployment_data['units'].str.lower()
unemployment_data['location'] = unemployment_data['location'].str.lower()
unemployment_data = unemployment_data.sort_values('state_indicator county_indicator msa_indicator title'.split(), ascending=[False, False, False, True])
unemployment_data.to_parquet('./fred_unemployment.parquet')
unemployment_data.head()

,title,units,location,msa_indicator,county_indicator,state_indicator,id
series id,,,,,,,
ALURN,unemployment rate in alabama,percent,alabama,0,0,1,ALURN
AKURN,unemployment rate in alaska,percent,alaska,0,0,1,AKURN
AZURN,unemployment rate in arizona,percent,arizona,0,0,1,AZURN
ARURN,unemployment rate in arkansas,percent,arkansas,0,0,1,ARURN
CAURN,unemployment rate in california,percent,california,0,0,1,CAURN


In [ ]:
test = 